In [ ]:
# import packages and functions
import os

import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

In [ ]:
from general_functions import read_gpkg_layers, show_layers_and_columns, store_data

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
# define relative paths
waterschap = "Scheldestromen"
path_Scheldestromen = "..\..\Data_preprocessed\Waterschappen\Scheldestromen\Scheldestromen.gpkg"
output_gpkg_path = "../../Data_postprocessed/Waterschappen/Scheldestromen"

In [ ]:
Scheldestromen = read_gpkg_layers(
    gpkg_path=path_Scheldestromen,
    variables=[
        "stuw",
        "gemaal",
        # 'afsluitmiddel',
        "duikersifonhevel",
        "hydroobject",
        # 'peilgebiedvigerend',
        # 'peilgebiedpraktijk',
        # 'peilafwijkinggebied',
        # 'streefpeil',
    ],
    engine="pyogrio",
)

# the peilgebieden and streefpeilen do not contain overlapping values. Scheldestromen has delivered additional files as shapes
Scheldestromen["peilgebied"] = gpd.read_file(
    "..\..\Data_preprocessed\Waterschappen\Scheldestromen\Praktijkpeilgebieden_20231204.shp"
)

In [ ]:
Scheldestromen["peilgebied"]["code"] = Scheldestromen["peilgebied"]["GPGIDENT"]
Scheldestromen["peilgebied"]["globalid"] = Scheldestromen["peilgebied"]["GLOBALID"]
Scheldestromen["peilgebied"]["waterhoogte"] = Scheldestromen["peilgebied"]["GPGZP"]
Scheldestromen["peilgebied"]["nen3610id"] = "dummy_nen3610id_peilgebied_" + Scheldestromen["peilgebied"].index.astype(
    str
)

Scheldestromen["streefpeil"] = gpd.GeoDataFrame()
Scheldestromen["streefpeil"]["waterhoogte"] = Scheldestromen["peilgebied"]["waterhoogte"]
Scheldestromen["streefpeil"]["globalid"] = Scheldestromen["peilgebied"]["globalid"]
Scheldestromen["streefpeil"]["geometry"] = np.nan


Scheldestromen["peilgebied"] = Scheldestromen["peilgebied"][["code", "nen3610id", "globalid", "geometry"]]

In [ ]:
# convert multiz points to points
Scheldestromen["stuw"].geometry = Scheldestromen["stuw"].centroid
Scheldestromen["gemaal"].geometry = Scheldestromen["gemaal"].centroid

# Scheldestromen

In [ ]:
# stuw
Scheldestromen["stuw"] = Scheldestromen["stuw"][["code", "nen3610id", "globalid", "geometry"]]

# gemaal
Scheldestromen["gemaal"] = Scheldestromen["gemaal"][["code", "nen3610id", "globalid", "geometry"]]
Scheldestromen["gemaal"]["code"] = "dummy_code_gemaal_" + Scheldestromen["gemaal"].index.astype(str)

# hydroobject
Scheldestromen["hydroobject"]["code"] = Scheldestromen["hydroobject"]["naam"]
Scheldestromen["hydroobject"] = Scheldestromen["hydroobject"][["code", "nen3610id", "globalid", "geometry"]]

# duikersifonhevel
Scheldestromen["duikersifonhevel"] = Scheldestromen["duikersifonhevel"][["code", "nen3610id", "globalid", "geometry"]]

In [ ]:
# pd.merge(left = Scheldestromen['peilgebiedpraktijk'],
#          right = Scheldestromen['streefpeil'],
#          left_on = 'globalid',
#          right_on = 'peilgebiedpraktijkid')

In [ ]:
Scheldestromen["stuw"] = Scheldestromen["stuw"][["code", "nen3610id", "globalid", "geometry"]]
Scheldestromen["gemaal"] = Scheldestromen["gemaal"][["code", "nen3610id", "globalid", "geometry"]]
Scheldestromen["hydroobject"] = Scheldestromen["hydroobject"][["code", "nen3610id", "globalid", "geometry"]]
Scheldestromen["duikersifonhevel"] = Scheldestromen["duikersifonhevel"][["code", "nen3610id", "globalid", "geometry"]]

In [ ]:
# Scheldestromen['peilgebiedpraktijk']['geometry'] = Scheldestromen['peilgebiedpraktijk'].buffer(distance = 0)
# Scheldestromen['peilafwijkinggebied']['geometry'] = Scheldestromen['peilafwijkinggebied'].buffer(distance = 0)

In [ ]:
# peilgebied = pd.merge(left = Scheldestromen['streefpeil'],
#                       right = Scheldestromen['peilgebiedpraktijk'],
#                       left_on = 'peilgebiedpraktijkid',
#                       right_on = 'globalid')

In [ ]:
# streefpeil = gpd.GeoDataFrame()
# streefpeil['waterhoogte'] = peilgebied['waterhoogte']
# streefpeil['globalid'] = peilgebied['peilgebiedpraktijkid']
# streefpeil['geometry'] = np.nan
# Scheldestromen['streefpeil'] = gpd.GeoDataFrame(streefpeil)

In [ ]:
# Scheldestromen['peilgebied'] = gpd.GeoDataFrame()
# Scheldestromen['peilgebied'][['code', 'nen3610id', 'globalid', 'geometry']] = peilgebied[['code', 'nen3610id_y', 'globalid_y', 'geometry_y']]

In [ ]:
# delete irrelvant data
variables = ["peilgebiedpraktijk", "peilgebiedvigerend", "peilafwijkinggebied"]

for variable in variables:
    if str(variable) in Scheldestromen:
        del Scheldestromen[variable]

### Check for the correct keys and columns

In [ ]:
show_layers_and_columns(waterschap=Scheldestromen)

In [ ]:
for i in range(len(Scheldestromen["peilgebied"])):
    if isinstance(Scheldestromen["peilgebied"].loc[i, "geometry"], Polygon):
        Scheldestromen["peilgebied"].loc[i, "geometry"].plot()

In [ ]:
merged = pd.merge(left=Scheldestromen["peilgebied"], right=Scheldestromen["streefpeil"], on="globalid")

merged[merged.waterhoogte.isna()]

### Store data

In [ ]:
# Check if the directory exists
if not os.path.exists(output_gpkg_path):
    # If it doesn't exist, create it
    os.makedirs(output_gpkg_path)

store_data(waterschap=Scheldestromen, output_gpkg_path=output_gpkg_path + "/Scheldestromen")

In [ ]:
Scheldestromen["hydroobject"]